<a href="https://colab.research.google.com/github/skaurl/P4-Model-Optimization/blob/main/Mask_Classfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 593kB 7.7MB/s 
     |████████████████████████████████| 1.1MB 15.7MB/s 
     |████████████████████████████████| 849kB 37.0MB/s 
     |████████████████████████████████| 122kB 44.2MB/s 
     |████████████████████████████████| 645kB 46.7MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 1.2MB 42.9MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 112kB 40.4MB/s 
     |████████████████████████████████| 686kB 40.6MB/s 
     |████████████████████████████████| 174kB 46.4MB/s 
     |████████████████████████████████| 6.3MB 44.1MB/s 
     |████████████████████████████████| 194kB 36.9MB/s 
     |████████████████████████████████| 358kB 38.6MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 38.2MB 141kB/s 


In [2]:
import tarfile

f = tarfile.open('/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/train.tar.gz')
f.extractall()
f.close()

In [3]:
!find . -type f -name '._*' -delete

In [4]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm
from glob import glob
import shutil
import cv2
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [5]:
os.makedirs('/content/train_dataset')
os.makedirs('/content/train_dataset/mask')
os.makedirs('/content/train_dataset/no_mask')
os.makedirs('/content/train_dataset/background')

os.makedirs('/content/val_dataset')
os.makedirs('/content/val_dataset/mask')
os.makedirs('/content/val_dataset/no_mask')
os.makedirs('/content/val_dataset/background')

In [6]:
img_info = pd.read_csv('/content/train/train.csv')

img_info_train, img_info_val = train_test_split(img_info, test_size=0.1, random_state=42)

In [7]:
for num in tqdm(range(len(img_info_train))):
    for name in os.listdir('/content/train/images/'+img_info_train.iloc[num,4]):
        original_path = '/content/train/images/'+img_info_train.iloc[num,4]+'/'+name
        destination_path = '/content/train_dataset/'

        if name[:4] == 'mask':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'inco':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'norm':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
            
            img = cv2.imread(original_path)[:128,:128]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,:128]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[-128:,:128]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_c_'+name,img)
            img = cv2.imread(original_path)[:128,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_d_'+name,img)
            img = cv2.imread(original_path)[-128:,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_e_'+name,img)
            img = cv2.imread(original_path)[:128,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_f_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_g_'+name,img)
            img = cv2.imread(original_path)[-128:,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_h_'+name,img)

100%|██████████| 2430/2430 [03:43<00:00, 10.88it/s]


In [8]:
for num in tqdm(range(len(img_info_val))):
    for name in os.listdir('/content/train/images/'+img_info_val.iloc[num,4]):
        original_path = '/content/train/images/'+img_info_val.iloc[num,4]+'/'+name
        destination_path = '/content/val_dataset/'

        if name[:4] == 'mask':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'inco':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'norm':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(128,128))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
            
            img = cv2.imread(original_path)[:128,:128]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,:128]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[-128:,:128]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_c_'+name,img)
            img = cv2.imread(original_path)[:128,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_d_'+name,img)
            img = cv2.imread(original_path)[-128:,192-64:192+64]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_e_'+name,img)
            img = cv2.imread(original_path)[:128,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_f_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_g_'+name,img)
            img = cv2.imread(original_path)[-128:,-128:]
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_h_'+name,img)

100%|██████████| 270/270 [00:24<00:00, 11.06it/s]


In [9]:
train_data = DataLoader.from_folder('/content/train_dataset')
val_data = DataLoader.from_folder('/content/val_dataset')

INFO:tensorflow:Load image with size: 70470, num_label: 3, labels: background, mask, no_mask.
INFO:tensorflow:Load image with size: 7830, num_label: 3, labels: background, mask, no_mask.


In [10]:
custom = image_classifier.ModelSpec(uri='https://tfhub.dev/google/imagenet/mobilenet_v1_025_128/classification/5')

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
     layers.experimental.preprocessing.Normalization(mean=0.450, variance=0.225)
    ]
)

In [12]:
model = image_classifier.create(train_data=train_data,
                                model_spec=custom,
                                validation_data=val_data,
                                batch_size=512,
                                epochs=50,
                                learning_rate=5e-4,
                                shuffle=True,
                                use_augmentation=data_augmentation
                                )

INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1001)              475801    
_________________________________________________________________
dropout (Dropout)            (None, 1001)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3006      
Total params: 478,807
Trainable params: 3,006
Non-trainable params: 475,801
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


137/137 [==============================] - 129s 799ms/step - loss: 0.9834 - accuracy: 0.7036 - val_loss: 0.5479 - val_accuracy: 0.8930
Epoch 2/50
137/137 [==============================] - 109s 802ms/step - loss: 0.7482 - accuracy: 0.7893 - val_loss: 0.4937 - val_accuracy: 0.9247
Epoch 3/50
137/137 [==============================] - 108s 796ms/step - loss: 0.6925 - accuracy: 0.8071 - val_loss: 0.4717 - val_accuracy: 0.9363
Epoch 4/50
137/137 [==============================] - 107s 787ms/step - loss: 0.6558 - accuracy: 0.8214 - val_loss: 0.4824 - val_accuracy: 0.9303
Epoch 5/50
137/137 [==============================] - 106s 775ms/step - loss: 0.6337 - accuracy: 0.8286 - val_loss: 0.4390 - val_accuracy: 0.9482
Epoch 6/50
137/137 [==============================] - 106s 780ms/step - loss: 0.6103 - accuracy: 0.8395 - val_loss: 0.4485 - val_accuracy: 0.9438
Epoch 7/50
137/137 [==============================] - 107s 784ms/step - loss: 0.5982 - accuracy: 0.8452 - val_loss: 0.4626 - val_accura

In [13]:
config = QuantizationConfig.for_dynamic()

In [14]:
model.export(export_dir='/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/', quantization_config=config)

INFO:tensorflow:Assets written to: /tmp/tmprlcmo8lv/assets


INFO:tensorflow:Assets written to: /tmp/tmprlcmo8lv/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpykr8fp_0/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpykr8fp_0/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/model.tflite


In [15]:
model.evaluate_tflite('/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/model.tflite', val_data)

INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


{'accuracy': 0.9616858237547893}